In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [ ]:
# from datasets import load_dataset

# math_dataset = load_dataset("OctoThinker/MegaMath-Web-Pro-Max", split = "train", streaming = True)
# next(iter(math_dataset))

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

{'text': 'The concept of Effective Health (EH) has been discussed, and a new formula has been derived to incorporate different damage types into EH calculations. The formula is a weighted average of damage-specific EH against the hits that a tank is looking at. \n\nTo calculate the change in EH for a given gear change, the following steps can be taken:\n1. Determine the change in stats between the two gear sets.\n2. Calculate the change in damage-specific EH for 100% physical damage mitigated by armor (P-NEH), 100% physical damage not mitigated by armor (Pn-NEH), 100% magical damage mitigated by resistance (M-NEH), and 100% magical damage not mitigated by resistance (Mn-NEH).\n3. Plug these values into the equation to find the plane where the change in EH is zero.\n4. Determine the operator for the inequality by plugging in a known point.\n5. Use the inequality to determine whether a given combination of damage types will result in an EH upgrade or downgrade.\n\nThe formula can be used

In [4]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", truncation = True)
# tokenized_dataset = math_dataset.map(lambda x: tokenizer(x["text"]))

# next(iter(tokenized_dataset))

Token indices sequence length is longer than the specified maximum sequence length for this model (973 > 512). Running this sequence through the model will result in indexing errors


{'text': 'The concept of Effective Health (EH) has been discussed, and a new formula has been derived to incorporate different damage types into EH calculations. The formula is a weighted average of damage-specific EH against the hits that a tank is looking at. \n\nTo calculate the change in EH for a given gear change, the following steps can be taken:\n1. Determine the change in stats between the two gear sets.\n2. Calculate the change in damage-specific EH for 100% physical damage mitigated by armor (P-NEH), 100% physical damage not mitigated by armor (Pn-NEH), 100% magical damage mitigated by resistance (M-NEH), and 100% magical damage not mitigated by resistance (Mn-NEH).\n3. Plug these values into the equation to find the plane where the change in EH is zero.\n4. Determine the operator for the inequality by plugging in a known point.\n5. Use the inequality to determine whether a given combination of damage types will result in an EH upgrade or downgrade.\n\nThe formula can be used

In [5]:
import time
import pandas as pd
import requests
import math
from pathlib import Path
from tqdm.notebook import tqdm

def fetch_issues (
        owner = "huggingface",
        repo = "datasets",
        num_issues = 10_000, 
        rate_limit = 5_000,
        issues_path = Path(".")
):
    '''
        Download all the issues from a GitHub repository:

        args:
        - owner: The individual/organization name of the repository
        - repo: The title of the repository
        - num_issues:
        - rate_limit: How many requests per hour
        - issues_path:

        returns:

    '''

    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok = True)

    batch = []
    all_issues = []
    per_page = 100 # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)
    
    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )